# Model


In [161]:
import shutil
import urllib.request
import os
import cv2
import numpy as np
import yaml
import gdown

from pathlib import Path
from typing import *

In [162]:
models_folder = Path("models")

models_folder.mkdir(parents=True, exist_ok=True)
(models_folder / "__init__.py").touch()

Each tracker must inherit from the `Tracker` class.


In [163]:
%%writefile model.py
import os


class Tracker:
    def __init__(self, model_name: str):
        self.model_name: str = model_name

    def initialize(self, image_file, box):
        """Initialize the tracker with the first frame and the bounding box."""
        pass

    def track(self, image_file):
        """Track the object in the next frame and return the new bounding box."""
        pass

Overwriting model.py


In [164]:
def clone_repo(model: str, url: str):
    """Clone a repo from GitHub and put it in the models folder."""
    path_zip: Path = models_folder / (model + ".zip")
    dest_folder: Path = models_folder / model

    if dest_folder.exists():
        return

    urllib.request.urlretrieve(url, path_zip)
    shutil.unpack_archive(path_zip, models_folder)

    try:
        (models_folder / (model + "-master")).rename(dest_folder)
    except:
        (models_folder / (model + "-main")).rename(dest_folder)

    path_zip.unlink()

    (dest_folder / "__init__.py").touch()

## [SiamSE](https://github.com/isosnovik/SiamSE)


In [165]:
model = "SiamSE"
repo_url = "https://github.com/ISosnovik/SiamSE/archive/refs/heads/master.zip"
clone_repo(model, repo_url)

weight_path = str(models_folder / "SiamSE" / "checkpoint_vot.pth")

if not os.path.exists(weight_path):
    weight_url = "https://drive.google.com/uc?id=1WQ-9_QE9Xk9wj52vVcEDIXY2NBTupAnZ"
    gdown.download(weight_url, output=weight_path)

In [166]:
%%writefile -a model.py

import os
import cv2
import yaml
import numpy as np
import models.SiamSE.lib.models.models as models

from pathlib import Path
from models.SiamSE.lib.tracker import SESiamFCTracker
from models.SiamSE.lib.utils import load_pretrain, cxy_wh_2_rect, convert_color_RGB
from typing import *

def get_SESIAMFC_tracker():
    
    def get_axis_aligned_bbox(bbox):
        """Convert bbox to [xc, yc, w, h] format"""
        x, y, w, h = bbox
        return [x + w / 2, y + h / 2, w, h]
    
    class SEsiamfc(Tracker):
        def __init__(self):
            super().__init__("SEsiamfc")

            path_SiamSE = Path('models') / 'SiamSE'
            model_pretrained_path: Path = path_SiamSE / 'checkpoint_vot.pth'
            config_path: Path = path_SiamSE / 'configs' / 'test.yaml'
            
            with open(config_path, 'r') as f:
                tracker_config: Dict = yaml.load(f.read(), Loader=yaml.FullLoader)

            # Prepare model
            net: Any = models.__dict__[tracker_config['MODEL']](padding_mode='constant')
            net = load_pretrain(net, model_pretrained_path)
            net = net.eval().cuda()

            # Prepare tracker
            tracker_config: Dict = tracker_config['TRACKER']['VOT2017']
            self.tracker = SESiamFCTracker(net, **tracker_config)

        def initialize(self, image_file, box):
            # Prepare image
            image = cv2.imread(image_file)
            image = convert_color_RGB(image)
            
            # Prepare box
            cx, cy, w, h = get_axis_aligned_bbox(box)
            target_pos = np.array([cx, cy])
            target_sz = np.array([w, h])

            self.tracker.init(image, target_pos, target_sz)

        def track(self, image_file):
            # Prepare image
            image = cv2.imread(image_file)
            image = convert_color_RGB(image)
            
            # Track
            target_pos, target_sz = self.tracker.track(image)

            # Prepare bbox
            return cxy_wh_2_rect(target_pos, target_sz)
    
    return SEsiamfc()

Appending to model.py


## [PyECO](https://github.com/StrangerZhang/pyECO)


In [167]:
model = "pyECO"
url = "https://github.com/StrangerZhang/pyECO/archive/refs/heads/master.zip"

clone_repo(model, url)

In [168]:
if not (models_folder / model / "eco" / "features" / "_gradient.cpython-310-x86_64-linux-gnu.so").exists():
    %cd models/pyECO/eco/features/
    !python setup.py build_ext --inplace
    %cd -

In [169]:
%%file -a model.py

def get_PyECO_tracker():
    os.sys.path.append('models/pyECO')
    from eco import ECOTracker
    
    class PyECO(Tracker):
        def __init__(self):
            super().__init__("PyECO")
            self.tracker = ECOTracker(True)

        def initialize(self, image_file, box):
            image = cv2.imread(image_file)
            self.tracker.init(image, box)

        def track(self, image_file):
            image = cv2.imread(image_file)
            extremities = self.tracker.update(image)
            bbox = [extremities[0], extremities[1], extremities[2] - extremities[0], extremities[3] - extremities[1]]
            return bbox
    
    return PyECO()

Appending to model.py


## [MixFormer](https://github.com/MCG-NJU/MixFormer)

In [170]:
model = "MixFormer"
url = "https://github.com/Sefray/MixFormer/archive/refs/heads/master.zip"

clone_repo(model, url)

weight_file = 'mixformer_online_22k.pth.tar'
weight_path = str(models_folder / "Mixformer" / weight_file)

if not os.path.exists(weight_path):
    weight_url = "https://drive.google.com/uc?id=1-UonqFXM-jKNnkJmN8lSEczeaKiNVxeh"
    gdown.download(weight_url, output=weight_path)

    %cd models/MixFormer/
    !python tracking/create_default_local_file.py --workspace_dir . --data_dir ./data --save_dir .
    %cd -

In [171]:
%%file -a model.py

def get_MixFormer_tracker():
    os.sys.path.append('models/MixFormer')
    from lib.test.tracker.mixformer import MixFormer as MF
    import importlib
    class MixFormer(Tracker):
        def __init__(self):
            super().__init__("MixFormer")
            
            tracker_params = {}
            tracker_params['model'] = 'mixformer_online_22k.pth.tar'
            tracker_params['max_score_decay'] = 1.0
            tracker_params['vis_attn'] = 0

            param_module = importlib.import_module('lib.test.parameter.mixformer')
            search_area_scale = None
            if tracker_params is not None and 'search_area_scale' in tracker_params:
                search_area_scale = tracker_params['search_area_scale']
            model = ''
            if tracker_params is not None and 'model' in tracker_params:
                model = tracker_params['model']
            params = param_module.parameters('baseline', model, search_area_scale)
            if tracker_params is not None:
                for param_k, v in tracker_params.items():
                    setattr(params, param_k, v)
            
            self.tracker = MF(params)

        def initialize(self, image_file, box):
            image = cv2.imread(image_file)
            info = {'init_bbox': box}
            self.tracker.initialize(image, info)

        def track(self, image_file):
            image = cv2.imread(image_file)
            info = self.tracker.track(image)
            return info['target_bbox']

    return MixFormer()

Appending to model.py


## [AAA](https://github.com/songheony/A3T)


In [172]:
model = "A3T"
url = "https://github.com/Leiyks/A3T/archive/refs/heads/master.zip"

clone_repo(model, url)

This tracker is a bagging tracker. It uses the tracks of other trackers to make its own predictions. In our testing, we will try to see if the trackers using all the trackers we are testing is better then them individually.


In [173]:
if not (models_folder / "A3T" / "pysot-toolkit").exists():
    %cd models/A3T

    # clone frameworks
    !git clone https://github.com/songheony/pytracking.git
    !git clone https://github.com/StrangerZhang/pysot-toolkit

    # install region
    %cd pysot-toolkit/pysot/utils/
    !python setup.py build_ext --inplace
    %cd ../../../../../


Some modification on the path used has to be performed.


In [174]:
%%writefile -a model.py

def get_AAA_tracker():
    import numpy as np
    class AAA(Tracker):
        def __init__(self):
            super().__init__("AAA")
            import models.A3T.algorithms.aaa as aaa
            self.experts = [
                get_SESIAMFC_tracker(),
                get_PyECO_tracker(),
                get_MixFormer_tracker(),
            ]
            self.n_experts = len(self.experts) # number of experts
            theta, gamma = 0.92, 11  # you can tune hyperparameters by running run_tuning.sh
            self.tracker = aaa.AAA(self.n_experts, mode="LOG_DIR", threshold=theta)


        def initialize(self, image_path, bbox):
            for i in range(self.n_experts):
                self.experts[i].initialize(image_path, bbox)
            self.tracker.initialize(image_path, bbox)

        def track(self, image_path):
            experts_result = np.zeros((self.n_experts, 4))
            for i in range(self.n_experts):
                experts_result[i, :] = self.experts[i].track(image_path)
            state, offline, weight = self.tracker.track(image_path, experts_result)
            return state
  
    return AAA()

Appending to model.py


# All


In [175]:
%%writefile -a model.py

def load_models() -> Dict[str, Tracker]:
    """Load all models
    Returns:
        Dict[str, Tracker]: Dict of models
    """
    ret: Dict[str, Tracker] = {}
    
    ret['SEsiamFC'] = get_SESIAMFC_tracker()
    ret['AAA'] = get_AAA_tracker()
    ret['PyECO'] = get_PyECO_tracker()
    ret['MixFormer'] = get_MixFormer_tracker()
    
    return ret

Appending to model.py


#
